# Manutenzione preventiva - DoS Attacks

In [5]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np
import warnings
import functions_notebook_new as fn
import functions_notebook_arinew as fan
from datetime import timedelta
from scipy.stats import entropy
import os


warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
### Estrazione features, creazione dataframe, scrittura dataframes su disco
levels_of_interest = ['frame', 'eth', 'tcp', 'arp', 'udp', 'modbus', 'icmp', 'ip']
# clean captures
clean_capture = 'captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json'

# ping flooding captures 1 min
pf_capture_1 = 'captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS15m-0,5h_1.json'
pf_capture_2 = 'captures\\captures2\\pingFloodDDoS\\eth2dump-pingFloodDDoS15m-0,5h_1.json'
pf_capture_3 = 'captures\\captures3\\pingFloodDDoS\\eth2dump-pingFloodDDoS15m-0.5h_1.json'
# nome dataframe su disco
dfclean_disk = ".\\compressed_datasets\\df_clean.gzip"
dfpf1_disk = ".\\compressed_datasets\\dfpf1.gzip"
dfpf2_disk = ".\\compressed_datasets\\dfpf2.gzip"
dfpf3_disk = ".\\compressed_datasets\\dfpf3.gzip"

if not os.path.isfile(dfclean_disk):
	packets_cc = fn.load_packets_from_file(clean_capture)
	features_cc = fn.extract_features_from_packets(packets_cc, levels_of_interest)
	df_cc = pd.DataFrame(features_cc)
	features_cc.clear()
	df_cc.to_pickle(dfclean_disk, compression='gzip')
else:
	df_cc = pd.read_pickle(dfclean_disk, compression='gzip')

if not os.path.isfile(dfpf1_disk):
	pkt_pf_15m_05h = fn.load_packets_from_file(pf_capture_1)
	features_pf_15m_05h = fn.extract_features_from_packets(pkt_pf_15m_05h, levels_of_interest)
	df = pd.DataFrame(features_pf_15m_05h)
	features_pf_15m_05h.clear()
	df.to_pickle(dfpf1_disk, compression='gzip')
else:
	df = pd.read_pickle(dfpf1_disk, compression='gzip')

if not os.path.isfile(dfpf2_disk):
	pkt_pf_15m_05h_2 = fn.load_packets_from_file(pf_capture_2)
	features_pf_15m_05h_2 = fn.extract_features_from_packets(pkt_pf_15m_05h_2, levels_of_interest)
	df1 = pd.DataFrame(features_pf_15m_05h_2)
	features_pf_15m_05h_2.clear()
	df1.to_pickle(dfpf2_disk, compression='gzip')
else:
	df1 = pd.read_pickle(dfpf2_disk, compression='gzip')

if not os.path.isfile(dfpf2_disk):
	pkt_pf_15m_05h_2 = fn.load_packets_from_file(pf_capture_2)
	features_pf_15m_05h_2 = fn.extract_features_from_packets(pkt_pf_15m_05h_2, levels_of_interest)
	df1 = pd.DataFrame(features_pf_15m_05h_2)
	features_pf_15m_05h_2.clear()
	df1.to_pickle(dfpf2_disk, compression='gzip')
else:
	df1 = pd.read_pickle(dfpf2_disk, compression='gzip')

if not os.path.isfile(dfpf3_disk):
	pkt_pf_15m_05h_3 = fn.load_packets_from_file(pf_capture_3)
	features_pf_15m_05h_3 = fn.extract_features_from_packets(pkt_pf_15m_05h_3, levels_of_interest)
	df2 = pd.DataFrame(features_pf_15m_05h_3)
	features_pf_15m_05h_3.clear()
	df2.to_pickle(dfpf2_disk, compression='gzip')
else:
	df2 = pd.read_pickle(dfpf2_disk, compression='gzip')


In [7]:
print(len(df_cc))
print(len(df))
print(len(df1))
print(len(df2))

35430
197778
212192
212241


In [8]:
fn.calcolo_features_binarie(df_cc)
fn.calcolo_features_binarie(df)
fn.calcolo_features_binarie(df1)
fn.calcolo_features_binarie(df2)

In [9]:
df_cc['frame.frame.time_utc'] = pd.to_datetime(df_cc['frame.frame.time_utc'])
df['frame.frame.time_utc'] = pd.to_datetime(df['frame.frame.time_utc'])
df1['frame.frame.time_utc'] = pd.to_datetime(df1['frame.frame.time_utc'])
df2['frame.frame.time_utc'] = pd.to_datetime(df2['frame.frame.time_utc'])

In [10]:
df_cc.sort_values(by='frame.frame.time_utc')
df.sort_values(by='frame.frame.time_utc')
df1.sort_values(by='frame.frame.time_utc')
df2.sort_values(by='frame.frame.time_utc')

,frame.frame.encap_type,frame.frame.time,frame.frame.time_utc,frame.frame.time_epoch,frame.frame.offset_shift,frame.frame.time_delta,frame.frame.time_delta_displayed,frame.frame.time_relative,frame.frame.number,frame.frame.len,...,nbdgm,udp,eth,tcp,data,ip,igmp,dhcp,ipv6,icmp
0,1,"Aug 8, 2018 19:37:05.555487000 ora legale Eur...",2018-08-08 17:37:05.555487+00:00,1533749825.555487000,0.000000000,0.000000000,0.000000000,0.000000000,1,66,...,0,0,1,1,0,1,0,0,0,0
1,1,"Aug 8, 2018 19:37:05.562744000 ora legale Eur...",2018-08-08 17:37:05.562744+00:00,1533749825.562744000,0.000000000,0.007257000,0.007257000,0.007257000,2,85,...,0,0,1,1,0,1,0,0,0,0
2,1,"Aug 8, 2018 19:37:05.773296000 ora legale Eur...",2018-08-08 17:37:05.773296+00:00,1533749825.773296000,0.000000000,0.210552000,0.210552000,0.217809000,3,60,...,0,0,1,1,0,1,0,0,0,0
3,1,"Aug 8, 2018 19:37:05.867520000 ora legale Eur...",2018-08-08 17:37:05.867520+00:00,1533749825.867520000,0.000000000,0.094224000,0.094224000,0.312033000,4,66,...,0,0,1,1,0,1,0,0,0,0
4,1,"Aug 8, 2018 19:37:05.872108000 ora legale Eur...",2018-08-08 17:37:05.872108+00:00,1533749825.872108000,0.000000000,0.004588000,0.004588000,0.316621000,5,85,...,0,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212236,1,"Aug 8, 2018 20:07:04.798017000 ora legale Eur...",2018-08-08 18:07:04.798017+00:00,1533751624.798017000,0.000000000,0.094252000,0.094252000,1799.242530000,212237,66,...,0,0,1,1,0,1,0,0,0,0
212237,1,"Aug 8, 2018 20:07:04.807663000 ora legale Eur...",2018-08-08 18:07:04.807663+00:00,1533751624.807663000,0.000000000,0.009646000,0.009646000,1799.252176000,212238,85,...,0,0,1,1,0,1,0,0,0,0
212238,1,"Aug 8, 2018 20:07:05.015777000 ora legale Eur...",2018-08-08 18:07:05.015777+00:00,1533751625.015777000,0.000000000,0.208114000,0.208114000,1799.460290000,212239,60,...,0,0,1,1,0,1,0,0,0,0
212239,1,"Aug 8, 2018 20:07:05.110697000 ora legale Eur...",2018-08-08 18:07:05.110697+00:00,1533751625.110697000,0.000000000,0.094920000,0.094920000,1799.555210000,212240,66,...,0,0,1,1,0,1,0,0,0,0


In [11]:
start_time = df['frame.frame.time_utc'].iloc[0]
end_time=start_time+timedelta(seconds=298.8)
end_time2 = end_time + timedelta(seconds=900)

In [12]:
start_time_2 = df1['frame.frame.time_utc'].iloc[0]
end_time_2=start_time_2+timedelta(seconds=299.16)
end_time2_2 = end_time_2 + timedelta(seconds=900)

In [13]:
start_time_3 = df2['frame.frame.time_utc'].iloc[0]
end_time_3=start_time_3+timedelta(seconds=240.59)
end_time2_3 = end_time_3 + timedelta(seconds=900)

In [14]:
df_cc1 = df[(df['frame.frame.time_utc'] >= start_time) & (df['frame.frame.time_utc'] < end_time)].copy()
df_attack = df[(df['frame.frame.time_utc'] >= end_time) & (df['frame.frame.time_utc'] <= end_time2)].copy()
df_cc2 = df[(df['frame.frame.time_utc'] > end_time2)].copy()

In [15]:
df1_cc1_2 = df1[(df1['frame.frame.time_utc'] >= start_time_2) & (df1['frame.frame.time_utc'] < end_time_2)].copy()
df1_attack_2 = df1[(df1['frame.frame.time_utc'] >= end_time_2) & (df1['frame.frame.time_utc'] <= end_time2_2)].copy()
df1_cc2_2 = df1[(df1['frame.frame.time_utc'] > end_time2_2)].copy()

In [36]:
df2_cc1_3 = df2[(df2['frame.frame.time_utc'] >= start_time_3) & (df2['frame.frame.time_utc'] < end_time_3)].copy()
df2_attack_3 = df2[(df2['frame.frame.time_utc'] >= end_time_3) & (df2['frame.frame.time_utc'] <= end_time2_3)].copy()
df2_cc2_3 = df2[(df2['frame.frame.time_utc'] > end_time2_3)].copy()

In [24]:
df_cc_aggregation = fan.calcola_features(df_cc)
df_cc1_aggregation = fan.calcola_features(df_cc1)
df_cc2_aggregation = fan.calcola_features(df_cc2)
df_att_aggregation = fan.calcola_features(df_attack)

In [40]:
df1_cc1_aggregation = fan.calcola_features(df1_cc1_2)
df1_cc2_aggregation = fan.calcola_features(df1_attack_2)
df1_att_aggregation = fan.calcola_features(df1_cc2_2)

In [41]:
df2_cc1_aggregation = fan.calcola_features(df2_cc1_3)
df2_cc2_aggregation = fan.calcola_features(df2_attack_3)
df2_att_aggregation = fan.calcola_features(df2_cc2_3)

In [43]:
df2_att_aggregation.head()

,eth_count,ip_count,arp_count,udp_count,tcp_count,mbtcp_count,icmp_count,pkt_count,per_eth_count,per_ip_count,...,std_iplen,entropy_iplen,bytes_per_timeunit,pkt_per_timeunit,modbus_response_count,modbus_request_count,modb_req_resp_fraction,icmp_request_count,icmp_response_count,icmp_req_resp_fraction
0,848,844,2,0,69,27,775,848,1.0,0.995283,...,6.845820,0.422980,5009.4,169.6,13,13,1.0,775.0,0.0,0.0
1,1082,1075,5,0,92,34,983,1082,1.0,0.993530,...,6.941924,0.450484,6389.6,216.4,16,16,1.0,983.0,0.0,0.0
2,1093,1080,10,0,98,34,982,1093,1.0,0.988106,...,7.285949,0.490553,6432.0,218.6,16,16,1.0,982.0,0.0,0.0
3,1066,1062,2,0,79,33,983,1066,1.0,0.996248,...,6.663504,0.389358,6280.0,213.2,16,16,1.0,983.0,0.0,0.0
4,1092,1083,6,1,100,34,982,1092,1.0,0.991758,...,7.176504,0.487564,6461.6,218.4,16,16,1.0,982.0,0.0,0.0


In [45]:
df2_cc1_aggregation.head()

,eth_count,ip_count,arp_count,udp_count,tcp_count,mbtcp_count,icmp_count,pkt_count,per_eth_count,per_ip_count,...,std_iplen,entropy_iplen,bytes_per_timeunit,pkt_per_timeunit,modbus_response_count,modbus_request_count,modb_req_resp_fraction,icmp_request_count,icmp_response_count,icmp_req_resp_fraction
0,82,76,2,1,75,31,0,82,1.0,0.926829,...,35.651169,1.577387,800.2,16.4,15,15,1.0,0.0,0.0,0.0
1,104,98,4,0,98,34,0,104,1.0,0.942308,...,15.635887,1.538367,932.8,20.8,16,16,1.0,0.0,0.0,0.0
2,103,96,4,2,94,34,0,103,1.0,0.932039,...,32.395413,1.648658,980.0,20.6,16,16,1.0,0.0,0.0,0.0
3,103,97,4,0,97,34,0,103,1.0,0.941748,...,15.704947,1.543445,924.8,20.6,16,16,1.0,0.0,0.0,0.0
4,88,81,4,1,80,33,0,88,1.0,0.920455,...,17.743205,1.569808,796.8,17.6,16,16,1.0,0.0,0.0,0.0


In [46]:
df2_cc2_aggregation.head()

,eth_count,ip_count,arp_count,udp_count,tcp_count,mbtcp_count,icmp_count,pkt_count,per_eth_count,per_ip_count,...,std_iplen,entropy_iplen,bytes_per_timeunit,pkt_per_timeunit,modbus_response_count,modbus_request_count,modb_req_resp_fraction,icmp_request_count,icmp_response_count,icmp_req_resp_fraction
0,76,70,4,1,69,27,0,76,1.0,0.921053,...,17.538784,1.587152,683.0,15.2,13,13,1.0,82.0,0.0,0.0
1,98,92,4,0,92,34,0,98,1.0,0.938776,...,16.064304,1.567602,884.8,19.6,16,16,1.0,981.0,0.0,0.0
2,105,98,4,0,98,34,0,105,1.0,0.933333,...,16.164289,1.550176,932.8,21.0,16,16,1.0,981.0,0.0,0.0
3,83,79,2,0,79,33,0,83,1.0,0.951807,...,15.821978,1.487818,775.2,16.6,16,16,1.0,983.0,0.0,0.0
4,106,98,4,1,98,34,0,106,1.0,0.924528,...,16.655615,1.560500,932.8,21.2,16,16,1.0,982.0,0.0,0.0


In [ ]:
prova